Calculate RayTransfer Matrix (RTM) with PHiX Fast Camera
===
To show the RTM values, we calculate a certain pixel's RTM.

In [ ]:
import datetime
from pathlib import Path
from textwrap import dedent

import numpy as np
from cherab.tools.raytransfer import RayTransferPipeline2D
from matplotlib import pyplot as plt
from raysect.core.scenegraph import print_scenegraph
from raysect.optical import World
from raysect.optical.observer import FullFrameSampler2D

from cherab.phix.machine import import_phix_mesh
from cherab.phix.observer import import_phix_camera
from cherab.phix.plasma import import_equilibrium
from cherab.phix.tools import profile_1D_to_2D
from cherab.phix.tools.raytransfer import import_phix_rtc
from cherab.phix.tools.visualize import show_phix_profiles

Create scene world
---

In [ ]:
# generate scene world
world = World()

# import phix equilibrium
eq = import_equilibrium()

# import phix machine configuration
mesh = import_phix_mesh(world, reflection=True)

# import phix raytrasfer cylinder object
rtc = import_phix_rtc(world, equilibrium=eq)

# import phix camera
camera = import_phix_camera(world)

Define Observer pipeline
---

In [ ]:
rtp = RayTransferPipeline2D(name="rtm")

# set camera's pipeline property
camera.pipelines = [rtp]

Set camera parameters
---
To focus one pixel, the `mask` array is defined.

In [ ]:
focus_pixel = (64, 192)
mask = np.zeros(camera.pixels, dtype=bool)
mask[focus_pixel[0] - 1, focus_pixel[1] - 1] = True

In [ ]:
camera.frame_sampler = FullFrameSampler2D(mask=mask)
camera.min_wavelength = 655.6
camera.max_wavelength = 656.8
camera.spectral_rays = 1
camera.spectral_bins = rtc.bins
camera.per_pixel_samples = 10
camera.lens_samples = 20

Execute ray-tracing
---

In [ ]:
camera.observe()

Visualize RayTransfer Matrix in $r - z$ plane
---

Convert RTM shape to $r-z$ poloidal 2D shape

In [ ]:
rtm_profile = profile_1D_to_2D(rtp.matrix.sum(axis=0).sum(axis=0), rtc=rtc)

Get rid of 0 value to show the RTM profile logarithumically

In [ ]:
rtm_min = rtm_profile[rtm_profile > 0.0].min()
rtm_profile[rtm_profile <= 0.0] = rtm_min * 0.9

In [ ]:
fig = plt.figure(dpi=150)
fig, axes = show_phix_profiles(
    rtm_profile,
    fig=fig,
    rtc=rtc,
    clabel="RTM [m$^3\\cdot$ str]",
    plot_mode="log",
    scientific_notation=False,
)